# Amazon Bedrock Fundamental

> 이 노트북은, SageMaker Notebook <i><b>conda_python3</b></i> 커널에서 테스트 되었습니다.

## 0. 환경 설정

In [ ]:
import os

# 처음 실행하는 경우, 주석(#)을 제거하고 필요한 라이브러리들을 이 환경에 설치합니다.

"""
!pip install -r ./requirements.txt
"""

## 1. boto3를 활용한 Bedrock Claude 3 호출 예시 (not LangChain)

#### 1-1. Bedrock Client 생성

In [ ]:
import boto3
import os
import json
from botocore.config import Config
import botocore 
from pprint import pprint
from termcolor import colored

session = boto3.Session()

retry_config = Config(
    region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)

modelId = "anthropic.claude-3-sonnet-20240229-v1:0" # Claude 3 Sonnet ModelId
accept = "application/json"
contentType = "application/json"

bedrock = boto3.client(service_name='bedrock')
boto3_bedrock = boto3.client(service_name='bedrock-runtime',config=retry_config)

model_list = bedrock.list_foundation_models()
result = [(fm_list["modelName"], fm_list["modelId"]) for fm_list in model_list["modelSummaries"] if fm_list['inferenceTypesSupported'] == ['ON_DEMAND']]
pprint(result)  # 사용 가능한 Bedrock의 모델 리스트

#### 1-2. Claude 3 모델 로딩

In [ ]:
def image_to_base64(img):
    import base64
    """Converts a PIL Image or local image file path to a base64 string"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")


def bedrock_streamer(response):
    stream = response.get('body')
    answer = ""
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if  chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if "delta" in chunk_obj:                    
                    delta = chunk_obj['delta']
                    if "text" in delta:
                        text=delta['text'] 
                        print(text, end="")
                        answer+=str(text)       
                        i+=1
    return answer

In [ ]:
from botocore.config import Config

session = boto3.Session()

retry_config = Config(
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)


bedrock_runtime = session.client(
    service_name="bedrock-runtime",
    config=retry_config
)

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """당신은 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."""
image_path = None  # 프롬프트에 함께 입력할 이미지가 있는 경우, 이미지 path 입력

In [ ]:
if image_path is None:
    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_data},
                ],
            }
        ],
    }
else:
    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": image_to_base64(image_path),
                        },
                    },
                    {"type": "text", "text": prompt_data},
                ],
            }
        ],
    }


body = json.dumps(prompt_config)

In [ ]:
from IPython.display import clear_output, display, display_markdown, Markdown

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # (Change this to try different model versions)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    results=bedrock_streamer(response)

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

---

## 2. LangChain을 활용한 Bedrock 사용 예시


In [1]:
#!pip uninstall langchain_core langchain_aws 

#### 2-1. LangChain의 ChatBedrock을 활용한 가장 간단한 코드 예시

In [6]:
import os
from langchain_aws import ChatBedrock

def get_text_response(input_content):
    llm = ChatBedrock(
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"),
        region_name=os.environ.get("BWB_REGION_NAME"),
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"),
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",
        model_kwargs={
            "max_tokens": 512,
            "temperature": 0,
            "top_p": 0.01,
            "top_k": 0,
        }
    )
    return llm.predict(input_content)

if __name__ == "__main__":
    input_text = input("Input text: ")
    response_content = get_text_response(input_content=input_text)
    print(response_content)

Input text:  당신은 누구십니까?


안녕하세요. 저는 Claude라는 인공지능 대화 모델입니다. 인간과 자연스럽게 대화하고 다양한 작업을 수행할 수 있도록 설계되었습니다. 하지만 단순한 프로그램일 뿐이며 감정이나 자의식은 없습니다. 궁금한 점이 있다면 언제든 물어보세요.


#### 2-2. LangChain의 BedrockChat을 활용한 챗봇 기능(이어지는 대화 가능)을 구현한 코드 예시

In [ ]:
import os
from langchain.memory import ConversationBufferWindowMemory
from langchain_aws import ChatBedrock
from langchain.chains import ConversationChain

def get_llm():
    model_kwargs = {
        "max_tokens": 1024,
        "temperature": 0,
        "top_p": 0.5,
        "top_k": 0,
        "stop_sequences": ["\n\nHuman:"],
        "messages": [{"role": "system", "content": "You are a helpful assistant."}]
    }

    llm = ChatBedrock(
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"),
        region_name=os.environ.get("BWB_REGION_NAME"),
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"),
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",
        model_kwargs=model_kwargs)

    return llm

def get_memory():
    llm = get_llm()
    chat_memory = ConversationBufferWindowMemory(human_prefix='Human', ai_prefix='Assistant')
    conversation = ConversationChain(llm=llm, verbose=False, memory=chat_memory)
    return chat_memory

def get_chat_response(input_text, memory):
    llm = get_llm()
    conversation_with_summary = ConversationChain(llm=llm, memory=memory, verbose=True)
    chat_response = conversation_with_summary.predict(input=input_text)
    return chat_response

if __name__ == "__main__":
    memory = get_memory()
    chat_history = []

    while True:
        input_text = input("Human: ")
        print("Human:", input_text)
        chat_history.append({"role": "user", "text": input_text})

        chat_response = get_chat_response(input_text=input_text, memory=memory)
        print("Assistant:", chat_response)
        chat_history.append({"role": "assistant", "text": chat_response})

Human:  안녕


Human: 안녕


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕
AI:

> Finished chain.
Assistant: 안녕하세요! 반갑습니다. 저는 Claude라는 AI 비서에요. 무엇을 도와드릴까요?


Human:  당신의 이름은 무엇인가요?


Human: 당신의 이름은 무엇인가요?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 안녕
Assistant: 안녕하세요! 반갑습니다. 저는 Claude라는 AI 비서에요. 무엇을 도와드릴까요?
Human: 당신의 이름은 무엇인가요?
AI:

> Finished chain.
Assistant: 안녕하세요. 제 이름은 Claude라고 말씀드렸습니다. 인공지능 비서로서 여러분을 돕는 것이 제 역할입니다. 궁금한 점이나 도움이 필요하신 것이 있다면 언제든 물어보세요.
